# Exploring Pydantic V2 Basics
### Lunch and Learn 
### 2024-05-02

This is a guided, hands-on intro to [Pydantic v2](https://docs.pydantic.dev/latest/).
If you're a Pydantic pro, this isn't for you. 
We could have a more advanced session with a Prefect Pydantic pro later, if there's interest.

![](./misc/handshake.gif)


## Prerequisites

1. Watch the Pydantic kick-off video introduction from Adam if you missed it.
1. Install Python 3.11 or 3.12 in a virtual environment and activate it.
1. Install Pydantic v2 in your virtual environment. `pip install -U pydantic jupyterlab`



In [ ]:
!pip install -U pydantic

Verify you have Pydantic >= 2.0.0 installed

In [42]:
# pip list pydantic  
# or
! conda list pydantic

# packages in environment at /opt/homebrew/Caskroom/miniforge/base:
#
# Name                    Version                   Build  Channel
pydantic                  2.7.1                    pypi_0    pypi
pydantic-core             2.18.2                   pypi_0    pypi
pydantic-settings         2.2.1                    pypi_0    pypi



## Context

Pydantic's upgrade from v1 and v2 has numerous breaking changes. See the Pydantic [migration guide](https://docs.pydantic.dev/2.6/migration/) for more information.

If you are interested in Prefect's internal transition to v2, join the #pydantic channel in Slack.


## Pydantic for validation

Pydantic has two primary ways to validate data.

You can use a function decorator or class-based models.

Let's try out the function decorator first.

### Function decorator
`validate_call` is a decorator (new name since V1)

#### Pros:
- quick to use - no definitions necessary

#### Cons:
- less flexible
- just for validation

In [43]:
def undecorated(a: int, b: int) -> int:
    print(a, b)


In [44]:

undecorated(3.0, "a")

3.0 a


In [45]:

from pydantic import validate_call

@validate_call()
def decorated(a: int, b: int) -> int:
    print(a, b)


In [46]:

decorated(3.0, "a")


ValidationError: 1 validation error for decorated
1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [47]:

from pydantic import validate_call

@validate_call()
def decorated(a: int, b: int) -> int:
    print(a, b)


In [48]:

decorated(3.0, "z")

ValidationError: 1 validation error for decorated
1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='z', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [49]:
decorated(3.0, "4")

3 4


What's happening here?


Can I get a strict mode?

Yes! Hat tip - Mason

In [85]:

from pydantic import validate_call

@validate_call(config={"strict": True})
def strict(a: int, b: int) -> int:
    print(a, b)

strict(3, 3)

3 3


In [86]:
from pydantic import validate_call

@validate_call(config={"strict": True})
def strict(a: int, b: int) -> int:
    print(a, b)

strict(3, "3")

ValidationError: 1 validation error for strict
1
  Input should be a valid integer [type=int_type, input_value='3', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type

### Validating return values

In [50]:
from pydantic import validate_call

@validate_call()
def decorated(a: int, b: int) -> int:
    print(a, b)
    return "abc"



In [51]:


decorated(3.0, "4")

3 4


'abc'

![](./misc/way.gif)

In [52]:
from pydantic import validate_call

@validate_call(validate_return=True)
def return_validated(a: int, b: int) -> int:
    print(a, b)
    return "abc"

return_validated(3, 4)

3 4


ValidationError: 1 validation error for return_validated
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [55]:
from pydantic import validate_call

@validate_call(validate_return=True)
def return_validated(a: int, b: int) -> int:
    print(a, b)
    return "7"

return_validated(3, "4")

3 4


7

# Pydantic models for schema creation and validation

![Zoolander pic of models](./misc/zoolander.png)


A Pydantic model is a class that inherits from Pydantic's `BaseModel` class.

Here's an example:

In [57]:
from datetime import datetime
from pydantic import BaseModel

class Delivery(BaseModel):
    timestamp: datetime
    dimensions: tuple[int, int]


m = Delivery(timestamp="2020-01-02T03:04:05Z", dimensions=["10", "20"])


In [58]:
print(m.timestamp)


2020-01-02 03:04:05+00:00


In [59]:
print(m.dimensions)

(10, 20)


In [60]:
print(type(m.timestamp))


<class 'datetime.datetime'>


🤯

In [61]:
print(type(m.dimensions[0]))



<class 'int'>



What just happened?






Try it again with different inputs.

In [62]:

 Delivery(timestamp="2020-01-02T03:04:05Z", dimensions=["z", "20"])


ValidationError: 1 validation error for Delivery
dimensions.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='z', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


Why did that happen?




Defined a schema for our data and Pydantic validated it for us.



Let's make another Pydantic model with a different schema.

Let's use a built-in Pydantic type for one of the fields.

In [63]:
from datetime import datetime
from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]



In [64]:

external_data = {
    "id": 123,
    "signup_ts": "2019-06-01 12:22",
    "tastes": {
        "wine": 9,
        b"cheese": 7,  # bytes literal
        "cabbage": "1",
    },
}


In [65]:

print(external_data)


{'id': 123, 'signup_ts': '2019-06-01 12:22', 'tastes': {'wine': 9, b'cheese': 7, 'cabbage': '1'}}


Let's make an instance of User with our dictionary.

In [66]:

user = User(
    **external_data
)  # Note that you need to unpack the dictionary here for instantiation of the User object


In [67]:
type(user)

__main__.User

In [68]:

print(user)


id=123 name='John Doe' signup_ts=datetime.datetime(2019, 6, 1, 12, 22) tastes={'wine': 9, 'cheese': 7, 'cabbage': 1}


 Note the coercion:

In [69]:

print(user.model_dump())  # Different method than Pydantic v1


{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


## Fields
The `Field` class is used to customize and add metadata to model fields.



###  Default values

In [70]:


from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(default="Person A")



In [71]:

user = User()

In [72]:
print(user)


name='Person A'



## JSON Schema generation with nested models

We can nest Pydantic models


In [73]:

import json
from enum import Enum

from typing import Annotated

from pydantic import BaseModel, Field
from pydantic.config import ConfigDict


class OptionalSizeExample(BaseModel):
    count: int
    size: float | None = None


class Gender(str, Enum):   # Enums are popular for form fields with multiple options
    male = "male"
    female = "female"
    other = "other"
    not_given = "not_given"


class MainModel(BaseModel):
    """
    This is the description of the main model
    """

    model_config = ConfigDict(title="Main")   # ConfigDict 

    size: OptionalSizeExample  
    gender: Annotated[Gender | None, Field(alias="Gender")] = None   # Annotated
    snap: int = Field(
        42,
        title="The Snap",
        description="this is the value of snap",
        gt=30,
        lt=50,
    )



In [74]:

main_model_schema = MainModel.model_json_schema() 


In [75]:
print(json.dumps(main_model_schema, indent=2))  


{
  "$defs": {
    "Gender": {
      "enum": [
        "male",
        "female",
        "other",
        "not_given"
      ],
      "title": "Gender",
      "type": "string"
    },
    "OptionalSizeExample": {
      "properties": {
        "count": {
          "title": "Count",
          "type": "integer"
        },
        "size": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Size"
        }
      },
      "required": [
        "count"
      ],
      "title": "OptionalSizeExample",
      "type": "object"
    }
  },
  "description": "This is the description of the main model",
  "properties": {
    "size": {
      "$ref": "#/$defs/OptionalSizeExample"
    },
    "Gender": {
      "anyOf": [
        {
          "$ref": "#/$defs/Gender"
        },
        {
          "type": "null"
        }
      ],
      "default": null
    },
    "s


This produces a "jsonable" dict of `MainModel`'s schema.

Calling json.dumps on the schema dict produces a JSON string.

What does this output remind you of? - Hint: something in the Prefect world.



Can customize the JSON schema output with the `json_schema_extra` option at field and/or model level
with Field or model_config

Field customization with `json_schema_extra` has built in parameters such as title, description, examples.


In [76]:

import json

from pydantic import BaseModel, EmailStr, Field, SecretStr


class User(BaseModel):
    age: int = Field(description="Age of the user")
    email: EmailStr = Field(examples=["marcelo@mail.com"])
    name: str = Field(title="Username")
    password: SecretStr = Field(
        json_schema_extra={
            "title": "Password",
            "description": "Password of the user",
            "examples": ["123456"],
        }
    )


print(json.dumps(User.model_json_schema(), indent=2))


{
  "properties": {
    "age": {
      "description": "Age of the user",
      "title": "Age",
      "type": "integer"
    },
    "email": {
      "examples": [
        "marcelo@mail.com"
      ],
      "format": "email",
      "title": "Email",
      "type": "string"
    },
    "name": {
      "title": "Username",
      "type": "string"
    },
    "password": {
      "description": "Password of the user",
      "examples": [
        "123456"
      ],
      "format": "password",
      "title": "Password",
      "type": "string",
      "writeOnly": true
    }
  },
  "required": [
    "age",
    "email",
    "name",
    "password"
  ],
  "title": "User",
  "type": "object"
}



## Built-in JSON parsing

Use `model_validate_json()`.

Fast (esp. in Pydantic 2.5 and greater).


In [77]:

from datetime import date

from pydantic import BaseModel, ConfigDict, ValidationError


class Event(BaseModel):
    model_config = ConfigDict(strict=True)

    when: date
    where: tuple[int, int]


json_data = '{"when": "1987-01-28", "where": [51, -1]}'  # string


In [78]:
print(Event.model_validate_json(json_data))

when=datetime.date(1987, 1, 28) where=(51, -1)


In [79]:

try:
    Event.model_validate_json('{"when": "1987-01-28", "where": ["51", -1]}')
except ValidationError as e:
    print(e)


1 validation error for Event
where.0
  Input should be a valid integer [type=int_type, input_value='51', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type


Does that give us what we expected?


## Serialization
Convert a model to a JSON-encoded string or a dictionary

### `model_dump()` method


In [80]:

print(user.model_dump())


{'name': 'Person A'}


In [81]:
print(type(user.model_dump()))


<class 'dict'>



### `model_dump_json()` method


In [82]:

print(user.model_dump_json())


{"name":"Person A"}


In [83]:
print(type(user.model_dump_json()))


<class 'str'>



❗️ Note the different output types.

Serialization generally, and serialization of nested models specifically, is a whole thing.
See the docs for the details: https://docs.pydantic.dev/latest/concepts/serialization/


## Recap

You've seen how to use Pydantic to validate data with function decorators and class-based models.

You've also seen how to serialize Pydantic models to dictionaries and strings.



We've just scratched the surface of what you can do with Pydantic. Play around with Pydantic v2 to go deeper and have fun building! 🏗️

![](./misc/fun.gif)